# Workflow #1

This notebook assesses and plots the AOIs.

In [ ]:

import algo.net_helper as nh
import os.path
import geopandas as gpd
import numpy as np
import contextily as cx
from matplotlib_scalebar.scalebar import ScaleBar
import matplotlib.pyplot as plt
import rasterio as rio

# settings
aoi_names = ["at_wien", "at_zs", "at_ib", "at_no", "at_zw", "at_graz_15"]

EDGE_WIDTH = 7000

file_suffix = ""

dir_data = "data"

mode = "bike_incwalk"
# tolerable access is determined by input network: all segments that have an index value assigned 
# (other than NULL, > 0) but have mode access set to False
access = "bicycle" 

# generated params
f_network_cent = os.path.join(dir_data, f"r_<aoi_name>_edges{file_suffix}.gpkg")
f_ps = os.path.join(dir_data, f"ps_<aoi_name>{file_suffix}.gpkg")

In [ ]:
cols = []

# first, get all columns (max. possible set)
for aoi in aoi_names:
    fn = f_network_cent.replace("<aoi_name>", aoi)
    if not os.path.isfile(fn):
        print("FILE NOT FOUND:", fn)
        continue
    print("reading file:", fn)
    cs = nh.read_cols_from_gpkg(fn)
    for c in cs:
        if not c in cols:
            cols.append(c)
print("found a total of", len(cols), "columns")

In [ ]:
# now, find columns not present for the given AOI
missing = dict([(k,[]) for k in cols])
for aoi in aoi_names:
    fn = f_network_cent.replace("<aoi_name>", aoi)
    if not os.path.isfile(fn):
        print("FILE NOT FOUND:", fn)
        continue
    print("reading file:", fn)
    cs = nh.read_cols_from_gpkg(fn)
    for c in cols:
        if not c in cs:
            missing[c].append(aoi)
            
print("\n--- RESULTS ---")
for k in missing:
    if k.endswith("_ft") or k.endswith("_tf"):
        continue
    v = missing[k]
    if len(v) > 0:
        print(k.replace("_sum", ""), v)

## Retrieve spatial extent

In [ ]:
import sqlite3
import pandas as pd
import geopandas as gpd
import shapely as sly
# get extent per AOI
target_crs = 32633

aois = []
for aoi in aoi_names:
    fn = f_network_cent.replace("<aoi_name>", aoi)
    if not os.path.isfile(fn):
        print("FILE NOT FOUND:", fn)
        continue
    print("reading file:", fn)
    con = sqlite3.connect(fn)
    srid, xmin, xmax, ymin, ymax = con.execute("SELECT srs_id, min_x, max_x, min_y, max_y FROM gpkg_contents LIMIT 1").fetchone()
    con.close()
    full_ext = sly.Polygon(((xmin, ymin), (xmin, ymax), (xmax, ymax), (xmax, ymin), (xmin, ymin)))
    aois.append({
        "aoi_name": aoi,
        "srid": srid
    })
aoi_df = pd.DataFrame(aois)
aoi_df

In [ ]:
for idx in aoi_df.index.values:
    # plot individual AOIs: map outer bounds: full_extent; marker: core_extent
    # basemap: OSM
    # overlay: network (very thin / hairline)
    aoi_name = aoi_df.loc[idx].aoi_name
    print(aoi_name)
    # load network geometry
    print("loading network geometry...")
    net = gpd.read_file(f_network_cent.replace("<aoi_name>", aoi_name), columns=["osm_id", "geometry"])
    # compute envelope on network reprojected to WGS84 (which was originally used to query data)
    orig_env_wgs84 = net.to_crs(4326).union_all().envelope
    orig_env = nh.transform_geom(orig_env_wgs84, 4326, aoi_df.loc[idx].srid, inv_xy=True)
    # assign precise AOI extents
    aoi_df.loc[idx, "full_extent"] = orig_env
    aoi_df.loc[idx, "core_extent"] = orig_env.buffer(-EDGE_WIDTH)
    # add reprojected versions (to target_crs)
    aoi_df.loc[idx, "full_extent_wgs84"] = orig_env_wgs84
    aoi_df.loc[idx, "core_extent_wgs84"] = nh.transform_geom(orig_env.buffer(-EDGE_WIDTH), aoi_df.loc[idx].srid, 4326, True)
    # compute x and y dimensions from original input
    orig_bounds_wgs84 = orig_env_wgs84.bounds
    p1 = sly.Point([orig_bounds_wgs84[0], orig_bounds_wgs84[1]])
    p2 = sly.Point([orig_bounds_wgs84[0], orig_bounds_wgs84[3]])
    p3 = sly.Point([orig_bounds_wgs84[2], orig_bounds_wgs84[1]])
    ## p1-p2: y-direction; p1-p3: x-direction
    p1t = nh.transform_geom(p1, 4326, target_crs, True)
    p2t = nh.transform_geom(p2, 4326, target_crs, True)
    p3t = nh.transform_geom(p3, 4326, target_crs, True)
    dx = p1t.distance(p3t)
    dy = p1t.distance(p2t)
    print("dx:", dx, "- dy:", dy, "|| dimensions in metric CRS:", orig_env.bounds[2]-orig_env.bounds[0], orig_env.bounds[3]-orig_env.bounds[1])
    ## append info to df
    aoi_df.loc[idx, "total_dim_x"] = dx
    aoi_df.loc[idx, "total_dim_y"] = dy
    aoi_df.loc[idx, "total_area"] = dx*dy
    aoi_df.loc[idx, "center_dim_x"] = dx-EDGE_WIDTH*2
    aoi_df.loc[idx, "center_dim_y"] = dy-EDGE_WIDTH*2
    aoi_df.loc[idx, "center_area"] = (dx-EDGE_WIDTH*2)*(dy-EDGE_WIDTH*2)
    
    # now, plot AOI
    print("plotting...")
    aoi_core_gdf = gpd.GeoDataFrame([aoi_df.loc[idx]], geometry="core_extent", crs=aoi_df.loc[idx].srid)
    aoi_full_gdf = gpd.GeoDataFrame([aoi_df.loc[idx]], geometry="full_extent", crs=aoi_df.loc[idx].srid)
    fext = aoi_df.loc[idx].full_extent.bounds
    pltdir_aoi = os.path.join("plots", aoi_name)
    if not os.path.exists(pltdir_aoi):
        os.makedirs(pltdir_aoi)
    ax = aoi_full_gdf.plot(facecolor="none", edgecolor="#B8014A55", linewidth=0.5, figsize=(5,5))
    #net.plot(ax=ax, linewidth=0.1, edgecolor="#555")
    aoi_core_gdf.plot(ax=ax, facecolor="#B8014A11", edgecolor="#B8014A", linewidth=0.5)
    ax.set_xlim([fext[0], fext[2]])
    ax.set_ylim([fext[1], fext[3]])
    cx.add_basemap(ax, crs=target_crs, zoom_adjust=1, source=cx.providers.OpenStreetMap.Mapnik, attribution="") # source=cx.providers.CartoDB.Positron
    ax.set_axis_off()
    ax.add_artist(ScaleBar(1))
    print("saving file...")
    plt.savefig(os.path.join(pltdir_aoi, "aoi.png"), dpi=600, bbox_inches='tight', pad_inches=0, transparent=True)

In [ ]:
aoi_df

In [ ]:
aoi_gdf = gpd.GeoDataFrame(aoi_df, geometry="full_extent_wgs84", crs=4326)
aoi_gdf[["aoi_name", "srid", "full_extent_wgs84"]].explore()

In [ ]:
pltdir = "plots"
if not os.path.exists(pltdir):
    os.makedirs(pltdir)

ax = aoi_gdf.full_extent_wgs84.to_crs(3857).plot(facecolor="#017AB822", edgecolor="#017AB8", linewidth=0.5, figsize=(20,20))
aoi_gdf.set_geometry("core_extent_wgs84", crs=4326).to_crs(3857).plot(ax=ax, facecolor="#017AB822", edgecolor="#017AB8", linewidth=0.5, figsize=(20,20))
cx.add_basemap(ax, crs=3857, source=cx.providers.CartoDB.Positron, zoom_adjust=1)

ax.set_axis_off()
ax.add_artist(ScaleBar(1))
plt.savefig(os.path.join(pltdir, f"aois.png"), dpi=500, bbox_inches='tight', pad_inches=0, transparent=True)

In [ ]:
cols_export = []
for c in aoi_gdf.columns:
    if c.find("extent") < 0 or c==aoi_gdf.geometry.name:
        cols_export.append(c)
aoi_gdf[cols_export].to_file(os.path.join(dir_data, "aois.gpkg"), layer=aoi_gdf.geometry.name)

In [ ]:
aoi_gdf.set_geometry("core_extent_wgs84", crs=4326, inplace=True)
cols_export = []
for c in aoi_gdf.columns:
    if c.find("extent") < 0 or c==aoi_gdf.geometry.name:
        cols_export.append(c)
aoi_gdf[cols_export].to_file(os.path.join(dir_data, "aois.gpkg"), layer=aoi_gdf.geometry.name)